# Gráfico de Ponqué por tamaño de hatos en el Bajo Sinú

In [ ]:
import os
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, Markdown
import plotly.graph_objects as go

# Directorio que contiene los archivos CSV
directory = "./"

# Obtener la lista de archivos CSV en el directorio
csv_files = [file for file in os.listdir(directory) if file.endswith(".csv")]

# Verificar si hay archivos CSV en el directorio
if len(csv_files) == 0:
    print("No se encontraron archivos CSV en el directorio especificado.")
    exit()

# Cargar los datos de los archivos CSV en un DataFrame
dfs = []

for file in csv_files:
    file_path = os.path.join(directory, file)
    try:
        df = pd.read_csv(file_path)
        dfs.append(df)
    except pd.errors.EmptyDataError:
        print(f"El archivo {file} está vacío y no se puede cargar.")

# Verificar si se cargaron datos en el DataFrame
if len(dfs) == 0:
    print("No se pudo cargar ningún archivo CSV con datos.")
    exit()

# Concatenar los DataFrames en uno solo
data = pd.concat(dfs)

# Filtrar y mostrar la tabla deseada
def filter_table(departamento, municipio):
    filtered_data = data.loc[(data['DEPARTAMENTO'].isin(departamento)) & (data['MUNICIPIO'].isin(municipio))]
    filtered_data = filtered_data[['MUNICIPIO', 'TOTAL BOVINOS', ' No DE FINCAS 1 A 50', ' No DE FINCAS 51 A 100',
                                   ' No DE FINCAS 101 A 500', ' No DE FINCAS 501 O MAS']]
    total_row = filtered_data.sum().to_frame().T
    filtered_data = pd.concat([filtered_data, total_row], ignore_index=True)
    display(Markdown(filtered_data.to_markdown()))

    # Generar la gráfica de pastel
    labels = ['No DE FINCAS 1 A 50', 'No DE FINCAS 51 A 100', 'No DE FINCAS 101 A 500', 'No DE FINCAS 501 O MAS']
    values = total_row[[' No DE FINCAS 1 A 50', ' No DE FINCAS 51 A 100', ' No DE FINCAS 101 A 500', ' No DE FINCAS 501 O MAS']]

    fig = go.Figure(data=[go.Pie(labels=labels, values=values)])
    fig.update_layout(title='Distribución de Fincas por Cantidad de Bovinos')
    fig.show()

# Obtener la lista de departamentos y municipios únicos
departamentos = data['DEPARTAMENTO'].unique().tolist()
municipios = data['MUNICIPIO'].unique().tolist()

# Crear los campos de selección múltiple para departamentos y municipios
departamento_dropdown = widgets.SelectMultiple(options=departamentos, description='Departamento:')
municipio_dropdown = widgets.SelectMultiple(options=municipios, description='Municipio:')

# Función de actualización del campo de selección de municipios basado en la selección de departamentos
def update_municipios(*args):
    selected_departamentos = departamento_dropdown.value
    filtered_municipios = data.loc[data['DEPARTAMENTO'].isin(selected_departamentos), 'MUNICIPIO'].unique().tolist()
    municipio_dropdown.options = filtered_municipios

# Asociar la función de actualización al evento de cambio de selección de departamentos
departamento_dropdown.observe(update_municipios, 'value')

# Mostrar los campos de selección
widgets.interact(filter_table, departamento=departamento_dropdown, municipio=municipio_dropdown)